In [2]:
%pylab widget
%load_ext autoreload
%autoreload 2

from analysis_functions import *
import fastcat.fastcat as fc
from scipy.optimize import minimize

Populating the interactive namespace from numpy and matplotlib


# First lets look at the GOS

In [3]:
# %%timeit -r 1
nviews = 493 

s = fc.Spectrum()
s.load('Varian_truebeam')

s.x[0] = 1; s.x[1] = 2
s.attenuate(0.,fc.get_mu(z=13)) #3.7

kernel = fc.Kernel(s,'CuGOS-784-micrometer')
kernel.add_focal_spot(1.2)
angles = np.linspace(0,np.pi*2,nviews)

phantom = fc.Catphan_404()
phantom.phan_map = ['air','G4_NYLON-8062','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Delrin_revised','CATPHAN_Acrylic','CATPHAN_Teflon_revised','air','CATPHAN_PMP','CATPHAN_Acrylic','CATPHAN_LDPE','CATPHAN_Acrylic','CATPHAN_Polystyrene','air','CATPHAN_Acrylic','air','CATPHAN_Teflon','air','air','air','air'] 
# phantom.phan_map = ['air','water','G4_NYLON-8062','G4_NYLON-8062','CATPHAN_Delrin','G4_NYLON-8062','CATPHAN_Teflon_revised','air','CATPHAN_PMP','G4_NYLON-8062','CATPHAN_LDPE','G4_NYLON-8062','CATPHAN_Polystyrene','air','CATPHAN_Acrylic'] 
phantom.return_projs(kernel,s,angles,mgy=1.1,bowtie=True,filter='FF0')
phantom.reconstruct('FDK',filt='ram_lak')

(19, 41, 41) 19
bowtie is off False
Using the bowtie filter True
really doing it!
ratio is 1210.184950271092 number of photons 0.02636808212587352


In [18]:
plt.figure()
plt.imshow((slice_recon),cmap = 'gray')#,vmax=0.10,vmin=0.03)

recon_slice = phantom.img[5]
im = create_mask(recon_slice.shape,r=5.7,radius=0.4)

contrast_fc, CNR_fc, noise_fc = return_CNR(recon_slice,im)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
return_CNR(recon_slice,im,show_map=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([10.87185137, 12.78895033, 13.29805794, 15.22699117, 17.57380394,
       23.79155838])

In [1270]:
plt.figure()
plt.imshow(np.where(im == 0, slice_recon,np.zeros_like(slice_recon)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1271]:
bcca_recon = np.load('/home/xcite/MV_CBCT/recon_6x_300_rared.npy')

plt.figure()
plt.imshow(np.rot90(bcca_recon.T[48],2),vmax=0.012,cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1272]:
recon_slice = np.rot90(np.mean(bcca_recon.T[35:43],0),2)
im = create_mask(recon_slice.shape,off =[-0.15,0], r= 5.75,radius=0.3)
contrast = []
CNR = []

for ii in range(2,int(np.max(im)+1)):
    
    contrast.append(np.mean(recon_slice[im == ii]))
    CNR.append(contrast[-1]/np.sqrt(np.std(recon_slice[im == ii])**2 + np.std(recon_slice[im == 6])**2))

bcca_data = np.array(sorted(contrast))
CNR_bcca = np.array(CNR)[np.argsort(contrast)]

In [16]:
def return_CNR(recon_slice,im,show_map=False):
    
    contrast = []
    CNR = []
    noise = []

    for ii in range(2,int(np.max(im)+1)):

        contrast.append(np.mean(recon_slice[im == ii]))
        CNR.append(contrast[-1]/np.sqrt(np.std(recon_slice[im == ii])**2 + np.std(recon_slice[im == 6])**2))
        noise.append(np.std(recon_slice[im == ii]))
        
    if show_map:
        plt.figure()
        plt.imshow(np.where(im==0,recon_slice,np.zeros_like(recon_slice)))

    return np.array(sorted(contrast)), np.array(CNR)[np.argsort(contrast)], np.array(noise)[np.argsort(contrast)]

In [1312]:
plt.figure()
plt.imshow(np.where(im == 0, recon_slice,np.zeros_like(slice_recon)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1274]:
plt.figure()
plt.plot(CNR_bcca)
plt.plot(CNR_fc,'r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Now lets load the experimental data which was generated in ~/MV_CBCT and saved as a npy file

In [1275]:
recon_slice = np.rot90(bcca_recon.T[45],2)
im = create_mask(recon_slice.shape,off =[-0.2,0], r= 5.8)
contrast = []
CNR = []

for ii in range(2,int(np.max(im)+1)):
    
    contrast.append(np.mean(recon_slice[im == ii]))
    CNR.append(contrast[-1]/np.sqrt(np.std(recon_slice[im == ii])**2 + np.std(recon_slice[im == 3])**2))

bcca_data = np.array(sorted(contrast))
CNR_data = np.array(CNR)[np.argsort(contrast)]

In [1311]:
def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

# bcca_data = bcca_data[np.arange(len(contrast)) !=6 ]
# cc = cc[np.arange(len(contrast)) != 6 ]

res = minimize(HU_fun,[cc[0],cc[5]+0.001],args=(cc,bcca_data))

x = [.853,0.945,.998,1.147,1.36,1.87]
fc_data = (cc - res.x[0])*(res.x[1])

plt.figure()
plt.plot(x,fc_data,'s',c='cornflowerblue',label='fastCAT')
plt.plot(x,bcca_data,'s',c='darkorange',label='Star-lack')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1288]:
np.save('mv_data_revised_delrin',fc_data)
np.save('mv_data_revised_delrin_bcca',bcca_data)

In [1029]:
# np.save('image_fc_mv_6x',im_fc)
# np.save('image_bcca_mv_6x',recon_slice)

In [1310]:
# phantom.img[4] = np.load('recons/6x_gos_recon_ram_lak_acrylic_493_valid_hybrid_true_phasespace.npy')
# phantom.img[4] = np.load('recons/6x_gos_recon_ram_lak_acrylic_493_valid_hybrid.npy')
plt.figure()
plt.subplot(121)
plt.imshow(recon_slice,cmap='gray',vmin=0.001,vmax=0.007)
plt.axis('off')
plt.subplot(122)
plt.imshow((phantom.img[4]- res.x[0])*(res.x[1]),cmap='gray',vmin=0.001,vmax=0.007)
plt.axis('off')
plt.tight_layout()

im_fc = (phantom.img[4]- res.x[0])*(res.x[1])
plt.figure()
plt.plot(recon_slice[256,:])
plt.plot(im_fc[256,:])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1031]:
hh = 4
plt.figure()
plt.imshow(np.abs(recon_slice[:,hh:] - ((phantom.img[4]- res.x[0])*(res.x[1]))[:,:-hh]),cmap='gray',vmin=0.00,vmax=0.001)
plt.axis('off')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1032]:
1-pp[1]/np.std(fc_data)**2

NameError: name 'pp' is not defined

In [1033]:
1-qq[1]/np.std(star_data)**2

NameError: name 'qq' is not defined

# Now lets look at the kV data 

In [1289]:
nviews = 887 # That is how many are in the file 

In [1318]:
%%timeit -r 1
s = fc.calculate_spectrum(100, 14, 3, 100,monitor=None)
s.attenuate(0.6,fc.get_mu(z=13))
s.attenuate(0.089,fc.get_mu(z=22))
kernel = fc.Kernel(s,'CsI-784-micrometer')
angles = np.linspace(0,np.pi*2,nviews)        

phantom = fc.Catphan_404()
phantom.phan_map = ['air','G4_NYLON-8062','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Delrin','CATPHAN_Acrylic','CATPHAN_Teflon_revised','air','CATPHAN_PMP','CATPHAN_Acrylic','CATPHAN_LDPE','CATPHAN_Acrylic','CATPHAN_Polystyrene','air','CATPHAN_Acrylic','air','CATPHAN_Teflon_revised','air','air','air','air'] 
phantom.return_projs(kernel,s,angles,scat_on=True,mgy=0.001,bowtie=True,filter='bowtie8')
phantom.reconstruct('FDK',filt='ram_lak')

(19, 19, 19) 19
bowtie is off False
Using the bowtie filter True
really doing it!
ratio is 18.63458183722777 number of photons 0.6662745014665195
(19, 19, 19) 19
bowtie is off False
Using the bowtie filter True
really doing it!
ratio is 18.63458183722777 number of photons 0.6662745014665195
51.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [1299]:
# im1 = phantom.img[4]

plt.figure()
plt.subplot(131)
plt.imshow((phantom.img[4]),cmap='gray')
plt.subplot(132)
plt.imshow((im1),cmap='gray')
plt.title('original bowtie')
plt.subplot(133)
plt.imshow((im1 - phantom.img[4]),cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1300]:
bcca_recon = np.load('/home/xcite/MV_CBCT/recon_kv_jan_2021.npy')

plt.figure()
# plt.subplot(121)
rr = plt.imshow(np.rot90(bcca_recon.T[40],3),cmap='gray',vmin=0.013,vmax=0.027)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1301]:
recon_slice = np.rot90(bcca_recon.T[50]).copy()
# im = create_mask(recon_slice.shape,off = [-0.1,-0.02], r= 4.45, radius = 0.3)
im = create_mask(recon_slice.shape,r=5.8,off = [0.2,0.05],rot = 1)
contrast = []
CNR = []
noise = []

for ii in range(2,int(np.max(im)+1)):
    contrast.append(np.mean(recon_slice[im == ii]))
    CNR.append(contrast[-1]/np.sqrt(np.std(recon_slice[im == ii])**2 + np.std(recon_slice[im == 3])**2))
    noise.append(np.std(recon_slice[im == ii]))
    
# Get rid of the delrin
bcca_noise = np.array(noise)[np.argsort(contrast)]
bcca_data = np.array(sorted(contrast))
CNR_data = np.array(CNR)[np.argsort(contrast)]

In [1302]:
recon_slice = phantom.img[4]
im = create_mask(recon_slice.shape,r=5.75,radius=0.4)
contrast = []
CNR = []
noise = []

for ii in range(2,int(np.max(im)+1)):
    contrast.append(np.mean(recon_slice[im == ii]))
    CNR.append(contrast[-1]/np.sqrt(np.std(recon_slice[im == ii])**2 + np.std(recon_slice[im == 3])**2))
    noise.append(np.std(recon_slice[im == ii]))

# Get rid of the delrin
cc = np.array(sorted(contrast))[np.arange(len(contrast))]
cc_CNR = np.array(CNR)[np.argsort(contrast)]
cc_noise = np.array(noise)[np.argsort(contrast)]

In [1303]:
plt.figure()
plt.imshow(np.where(im == 0,recon_slice,np.zeros_like(recon_slice)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1304]:
def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

# bcca_data = bcca_data[np.arange(len(contrast)) !=6 ]
# cc = cc[np.arange(len(contrast)) != 6 ]
res = minimize(HU_fun,[cc[0],cc[3]],args=(cc,bcca_data),method='Powell')
# x = [0,0,.853,0.945,.998,1.147,1.87]
            
fc_data = (cc - res.x[0])*(res.x[1])

fc_image = (recon_slice - res.x[0])*(res.x[1])

plt.figure()
plt.imshow(np.rot90(fc_image,0),cmap='gray',vmin=0.013,vmax=0.027)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1307]:
x = [.853,0.945,.998,1.147,1.36,1.87]
plt.figure()
plt.plot(x,bcca_data,'s',c='darkorange',label='')
plt.plot(x,fc_data,'*',c='cornflowerblue',label='fastCAT')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1170]:
# np.save('contrast_bcca_kv',bcca_data)

# Now lets load the experimental kV data which was generated in ~/MV_CBCT and saved as a npy file

In [1309]:
np.save('fc_kv_image_nylon',np.rot90(fc_image,0))

In [1308]:
plt.figure()
plt.plot(np.rot90(np.mean(bcca_recon.T[25:36],0),3)[256,:])
plt.plot(np.rot90(fc_image,0)[256,:])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

plt.figure()
recon_slice[im != 0] = 0
plt.imshow(recon_slice)

plt.figure()
recon_slice[im != 0] = 0
plt.imshow(recon_slice)

In [1103]:
plt.figure()
plt.plot(CNR_data)
plt.plot(cc_CNR,'r')
# plt.plot(CNR_fc,'r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1040]:
plt.figure()
plt.plot(1000*cc_noise)
plt.plot(bcca_noise,'r')
# plt.plot(CNR_fc,'r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
def HU_fun(a,x,y):
    return np.mean(np.abs((x - a[0])/(a[1] - a[0])*-1000 +1000 - y))

bcca_revised = bcca_data
cc2 = cc

res = minimize(HU_fun,[cc2[3],0.1*cc2[0]],args=(cc2,bcca_revised))

x = [0,0,.853,0.945,.998,1.147,1.87]
            
fc_data = (cc2 - res.x[0])/(res.x[1] - res.x[0])*-1000 + 1000
x = [0,0,.853,0.945,.998,1.147,1.36,1.87]
plt.figure()
plt.plot(x,bcca_revised,'s',c='darkorange',label='Star-lack')
plt.plot(x,fc_data,'r*',c='cornflowerblue',label='fastCAT')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [68]:
fc_data = (cc - res.x[0])/(res.x[1] - res.x[0])*-1000

xx = np.linspace(0,max(x),100)
pp = np.polyfit(x,fc_data,1,full=1)
qq = np.polyfit(x,bcca_data,1,full=1)
yy = np.polyval(pp[0],xx)
zz = np.polyval(qq[0],xx)

plt.ylabel('HU')
plt.xlabel('Relative electron density')
plt.title('CWO')
plt.xticks(x,['air1','air2','PMP','LDPE','Poly','Acrylic','Delrin','Teflon'],rotation=75)
plt.plot(xx,yy,':',c='cornflowerblue',lw=0.8)
plt.plot(xx,zz,'-.',c='darkorange',lw = 0.8)
plt.legend()
plt.tight_layout()
plt.savefig('initial_cwo')

In [172]:
plt.figure()
scale, offset, offset2 = 16.5, 0.00650, 0.057
plt.plot(scale*(bcca_data - offset),'rx')
plt.plot(cc - offset2,'kx')
plt.xlabel('Machine Output Value [??]')
plt.ylabel('Material')
plt.legend(['BCCA data','fastCAT'])
plt.title('Experimental Validation 6 MV 100MU')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Experimental Validation 6 MV 100MU')

In [153]:
1-pp[1]/np.std(fc_data)**2

array([0.99967224])

In [154]:
1-qq[1]/np.std(star_data)**2

array([1.])

In [46]:
av = 1/3*8.175600196835831e-10 + 2/3*4.769697969919662e-10

In [47]:
av

5.904998712225051e-10

In [49]:
7.199436602434162e-10/av

1.2192105287896593